# Message passing - control paths

In [1]:
import Pkg;
Pkg.activate("."); 
Pkg.instantiate();

  Activating project at `~/syndr/Wouter/Onderzoek/Projecten/tue/mp-controlpaths`


In [2]:
using Revise
using RxInfer
using Optim
using Plots
default(label="")

includet("Robots.jl"); using. Robots
includet("util.jl")

In [7]:
# Time
Δt = 0.5
len_trial = 10
tsteps = range(0, step=Δt, length=len_trial)
len_horizon = 5;

# Dimensions
Dy = 2
Dx = 4
Du = 2

# State transition
A = [1. 0. Δt 0.;
     0. 1. 0. Δt;
     0. 0. 1. 0.;
     0. 0. 0. 1.]

# Control matrix
B = [0. 0.;
     0. 0.;
     Δt 0.;
     0. Δt]

# Process noise covariance matrix
σ = [1e-2; 1e-2]
Q = [Δt^3/3*σ[1]            0.0    Δt^2/2*σ[1]            0.0;
     0.0            Δt^3/3*σ[2]            0.0    Δt^2/2*σ[2];
     Δt^2/2*σ[1]            0.0         Δt*σ[1]           0.0;
     0.0            Δt^2/2*σ[2]            0.0        Δt*σ[2]]

# Measurement function     
C = [1. 0. 0. 0.;
     0. 1. 0. 0.]

# Measurement noise covariance matrix
ρ = [1e-2, 1e-2]
R = diagm(ρ)

# Control prior precision
η = 0.0

# State prior parameters
m_0 = [0.0, -1., 0., 0.]
S_0 = 0.5diagm(ones(4));

In [8]:
@model function static_control_priors(y_k, u_k, A,B,Q,C,R,T, m_kmin1, S_kmin1)
    
    # Transition function 
    x_k ~ MvNormalMeanCovariance(A*m_kmin1 + B*u_k, Q + A*S_kmin1*A')
    
    # Measurement function
    y_k ~ MvNormalMeanCovariance(C*x_k, R)

    x_tmin1 = x_k
    for k in 1:T

        # Control prior
        u_[t] ~ MvNormalMeanCovariance(zeros(2), η*I(2))
        
        # Transition function 
        x_[t] ~ MvNormalMeanCovariance(A*x_tmin1 + B*u_[t], Q)
        
        # Measurement function
        y_[t] ~ MvNormalMeanCovariance(C*x_[t], R)

        x_tmin1 = x_[t]

    end
end

In [12]:
# Initial state
x_0 = [0.0, 0., 0., 0.]

# Setpoint (desired observation)
x_star = [1.0, 1.0, 0.0, 0.0]
goal = (C*x_star, 0.5*I(Dy))

# Initialize robot
control_lims = (-1., 1.)
bot = FieldBot(ρ,σ, Δt=Δt, control_lims=control_lims)

y_sim = zeros(Dy, len_trial)
x_sim = zeros(Dx, len_trial)
u_sim = zeros(Du, len_trial)

x_kmin1 = x_0
m_kmin1 = zeros(Dx)
S_kmin1 = 1e5*I(Dx)

for k in 1:len_trial

    # Evolve system
    y_sim[:,k], x_sim[:,k] = update(bot, x_kmin1, u_sim[:,k])
    x_kmin1 = x_sim[:,k]

    # Agent absorbs observation
    res = infer(
        model = static_control_priors(A=A,B=B,C=C,Q=Q,R=R,T=len_horizon),
        data = (y_k = y_sim[:,k], 
                u_k = u_sim[:,k],
                m_kmin1 = m_kmin1,
                S_kmin1 = S_kmin1),
    )

    # Extract state posterior
    m_kmin1 = mean(res.posteriors[:x_k])
    S_kmin1 = cov(res.posteriors[:x_k])

    # Extract control posterior
    u_sim[:,k] = res.posteriors[:u_1]

end


ErrorException: Model static_control_priors is not defined for 10 interfaces ((:A, :B, :C, :Q, :R, :T, :y_k, :u_k, :m_kmin1, :S_kmin1)).

In [10]:
# We are going to use some private functionality from ReactiveMP, 
# in the future we should expose a proper API for this
import RxInfer.ReactiveMP: getrecent, messageout

function create_agent(;T = 20, Fg, Fa, Ff, engine_force_limit, x_target, initial_position, initial_velocity)
    Epsilon = fill(huge, 1, 1)                # Control prior variance
    m_u = Vector{Float64}[ [ 0.0] for k=1:T ] # Set control priors
    V_u = Matrix{Float64}[ Epsilon for k=1:T ]

    Sigma    = 1e-4*diageye(2) # Goal prior variance
    m_x      = [zeros(2) for k=1:T]
    V_x      = [huge*diageye(2) for k=1:T]
    V_x[end] = Sigma # Set prior to reach goal at t=T

    # Set initial brain state prior
    m_s_t_min = [initial_position, initial_velocity] 
    V_s_t_min = tiny * diageye(2)
    
    # Set current inference results
    result = nothing

    # The `infer` function is the heart of the agent
    # It calls the `RxInfer.inference` function to perform Bayesian inference by message passing
    compute = (upsilon_t::Float64, y_hat_t::Vector{Float64}) -> begin
        m_u[1] = [ upsilon_t ] # Register action with the generative model
        V_u[1] = fill(tiny, 1, 1) # Clamp control prior to performed action

        m_x[1] = y_hat_t # Register observation with the generative model
        V_x[1] = tiny*diageye(2) # Clamp goal prior to observation

        data = Dict(:m_u       => m_u, 
                    :V_u       => V_u, 
                    :m_x       => m_x, 
                    :V_x       => V_x,
                    :m_s_t_min => m_s_t_min,
                    :V_s_t_min => V_s_t_min)
        
        model  = mountain_car(T = T, Fg = Fg, Fa = Fa, Ff = Ff, engine_force_limit = engine_force_limit) 
        result = infer(model = model, data = data)
    end
    
    # The `act` function returns the inferred best possible action
    act = () -> begin
        if result !== nothing
            return mode(result.posteriors[:u][2])[1]
        else
            return 0.0 # Without inference result we return some 'random' action
        end
    end
    
    # The `future` function returns the inferred future states
    future = () -> begin 
        if result !== nothing 
            return getindex.(mode.(result.posteriors[:s]), 1)
        else
            return zeros(T)
        end
    end

    # The `slide` function modifies the `(m_s_t_min, V_s_t_min)` for the next step
    # and shifts (or slides) the array of future goals `(m_x, V_x)` and inferred actions `(m_u, V_u)`
    slide = () -> begin

        model  = RxInfer.getmodel(result.model)
        (s, )  = RxInfer.getreturnval(model)
        varref = RxInfer.getvarref(model, s) 
        var    = RxInfer.getvariable(varref)
        
        slide_msg_idx = 3 # This index is model dependend
        (m_s_t_min, V_s_t_min) = mean_cov(getrecent(messageout(var[2], slide_msg_idx)))

        m_u = circshift(m_u, -1)
        m_u[end] = [0.0]
        V_u = circshift(V_u, -1)
        V_u[end] = Epsilon

        m_x = circshift(m_x, -1)
        m_x[end] = x_target
        V_x = circshift(V_x, -1)
        V_x[end] = Sigma
    end

    return (compute, act, slide, future)    
end

create_agent (generic function with 1 method)

Now it's time to see if we can help our friends arrive at the camping site by midnight?

In [8]:
(execute_ai, observe_ai) = create_world(
    Fg = Fg, Ff = Ff, Fa = Fa, 
    initial_position = initial_position, 
    initial_velocity = initial_velocity
) # Let there be a world

T_ai = 50

(compute_ai, act_ai, slide_ai, future_ai) = create_agent(; # Let there be an agent
    T  = T_ai, 
    Fa = Fa,
    Fg = Fg, 
    Ff = Ff, 
    engine_force_limit = engine_force_limit,
    x_target = x_target,
    initial_position = initial_position,
    initial_velocity = initial_velocity
) 

N_ai = 100

# Step through experimental protocol
agent_a = Vector{Float64}(undef, N_ai) # Actions
agent_f = Vector{Vector{Float64}}(undef, N_ai) # Predicted future
agent_x = Vector{Vector{Float64}}(undef, N_ai) # Observations

for t=1:N_ai
    agent_a[t] = act_ai()               # Invoke an action from the agent
    agent_f[t] = future_ai()            # Fetch the predicted future states
    execute_ai(agent_a[t])              # The action influences hidden external states
    agent_x[t] = observe_ai()           # Observe the current environmental outcome (update p)
    compute_ai(agent_a[t], agent_x[t]) # Infer beliefs from current model state (update q)
    slide_ai()                          # Prepare for next iteration
end

animation_ai = @animate for i in 1:N_ai
    # pls - plot landscape
    pls = plot(valley_x, valley_y, title = "Active inference results", label = "Landscape", color = "black")
    pls = scatter!(pls, [agent_x[i][1]], [height(agent_x[i][1])], label="car")
    pls = scatter!(pls, [x_target[1]], [height(x_target[1])], label="goal")   
    pls = scatter!(pls, agent_f[i], height.(agent_f[i]), label = "Predicted future", alpha = map(i -> 0.5 / i, 1:T_ai))
    
    # pef - plot engine force
    pef = plot(Fa.(agent_a[1:i]), title = "Engine force (agents actions)", xlim = (0, N_ai), ylim = (-0.05, 0.05))
    
    plot(pls, pef, size = (800, 400))
end
    
# The animation is saved and displayed as markdown picture for the automatic HTML generation
gif(animation_ai, "../pics/ai-mountain-car-ai.gif", fps = 24, show_msg = false); 

![](../pics/ai-mountain-car-ai.gif)